In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

In [2]:
# load vsech nasich + Honzovo dat o pozicich autobusu a spojeni do velke tabulky, odstaneni radku, kde je delay = NaN
data_nase = pd.read_csv('out.c-cleaning.ALL_POSITIONS_NEW.csv', encoding='utf-8', low_memory=False)
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', encoding='utf-8', low_memory=False)
data_wkend = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', encoding='utf-8', low_memory=False)
data_wkend_nase = pd.read_csv('out_clean_vikend_16_17.csv', encoding='utf-8', low_memory=False)

data = pd.concat([data_nase, data_Honza, data_wkend,data_wkend_nase], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')
data.head()

,UNICORN,TRIP_ID,START_TIME,HEADSIGN,DATE,BEARING,DELAY,DELAY_STOP_ARRIVAL,DELAY_STOP_DEPARTURE,IS_CANCELED,...,LAST_STOP_SEQUENCE,NEXT_STOP_ARRIVAL_TIME,NEXT_STOP_DEPARTURE_TIME,NEXT_STOP_ID,NEXT_STOP_SEQUENCE,ORIGIN_TIMESTAMP,SHAPE_DIST_TRAVELED,SPEED,LATITUDE,LONGITUDE
0,335_42_200316_19_May,335_42_200316,18:20:00,"Kamenice,Kult.dům",19 May,82,65.0,NaN,85.0,False,...,28.0,"Tue, 19 May 2020 19:13:00 Z","Tue, 19 May 2020 19:13:00 Z",U1564Z1,29.0,"Tue, 19 May 2020 19:13:17 Z",30.1,52,49.90354,14.58532
1,335_353_191111_19_May,335_353_191111,18:28:00,"Praha,Budějovická",19 May,313,-70.0,NaN,-32.0,False,...,26.0,"Tue, 19 May 2020 19:22:00 Z","Tue, 19 May 2020 19:17:00 Z",U488Z7,27.0,"Tue, 19 May 2020 19:15:06 Z",24.9,52,50.02853,14.45633
2,335_353_191111_19_May,335_353_191111,18:28:00,"Praha,Budějovická",19 May,358,-62.0,NaN,-57.0,False,...,27.0,"Tue, 19 May 2020 19:22:00 Z","Tue, 19 May 2020 19:22:00 Z",U50Z84,28.0,"Tue, 19 May 2020 19:16:13 Z",25.3,34,50.03116,14.45503
3,335_42_200316_19_May,335_42_200316,18:20:00,"Kamenice,Kult.dům",19 May,2,76.0,NaN,85.0,False,...,28.0,"Tue, 19 May 2020 19:13:00 Z","Tue, 19 May 2020 19:13:00 Z",U1564Z1,29.0,"Tue, 19 May 2020 19:12:54 Z",29.8,51,49.90253,14.58289
4,335_42_200316_19_May,335_42_200316,18:20:00,"Kamenice,Kult.dům",19 May,61,91.0,NaN,130.0,False,...,27.0,"Tue, 19 May 2020 19:13:00 Z","Tue, 19 May 2020 19:11:00 Z",U1565Z2,28.0,"Tue, 19 May 2020 19:11:59 Z",29.2,54,49.90037,14.57858


In [36]:
# pridani sloupce s route_name
data['ROUTE_NAME'] = data['TRIP_ID'].str[:3]

In [ ]:
# prevod sloupecku DATE na datovy format
data['DATE'] = data['DATE'] + " 2020"

In [5]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [6]:
# vytvoreni noveho sloupce se dnem v tydnu
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

In [60]:
# filtrace pouze linky 333, v ranni spicce, smer centrum -> novy mensi dataframe -> ranni_333
ranni_333 = data.loc[(data['HEADSIGN'] == 'Praha,Kačerov') & (data['ROUTE_NAME'] == '333') & (data['START_TIME'] < '11:00:00'), ['UNICORN', 'TRIP_ID', 'START_TIME', 'HEADSIGN', 'DATE',
       'DELAY', 'DELAY_STOP_ARRIVAL', 'DELAY_STOP_DEPARTURE',
       'ORIGIN_TIMESTAMP', 'SHAPE_DIST_TRAVELED', 'SPEED', 'LATITUDE',
       'LONGITUDE','ROUTE_NAME','DAY_OF_WEEK']]

# pocet unikatnich spoju -> 195
len(ranni_333['UNICORN'].unique().tolist())

195

In [9]:
# pocet unikatnich trip_id v souboru 
len(ranni_333['TRIP_ID'].unique())

21

In [42]:
# DataFrame se vsemi jizdnimi rady (pro unikatnich 21 trip_id) -> vytvoreno v jizdni_rad.ipynb
info_stops_clean = pd.read_csv('jizdnirad.csv', encoding='utf-8', low_memory=False)

In [78]:
info_stops_clean

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,STOP.GEOMETRY.COORDINATES,STOP.PROPERTIES.STOP_ID,STOP.PROPERTIES.STOP_NAME,NEJBLIZSI_POZICE,DELAY_STOP_DEPARTURE,DAY_OF_WEEK,DATE
0,9:18:00,9:18:00,0.00000,U1503Z2,1,333_677_200302,"[14.45708, 49.96319]",U1503Z2,"Dolní Břežany,Obecní úřad",0.0,NaN,Wednesday,2020-05-06
1,9:19:00,9:19:00,0.15836,U1502Z2,2,333_677_200302,"[14.45889, 49.96373]",U1502Z2,"Dolní Břežany,Náměstí",0.0,NaN,Wednesday,2020-05-06
2,9:21:00,9:21:00,0.77992,U1501Z2,3,333_677_200302,"[14.46199, 49.96894]",U1501Z2,"Dolní Břežany,Na kopečku",0.9,61.0,Wednesday,2020-05-06
3,9:25:00,9:25:00,3.53721,U545Z2,4,333_677_200302,"[14.46853, 49.99163]",U545Z2,Písnice,3.4,85.0,Wednesday,2020-05-06
4,9:26:00,9:26:00,3.91690,U546Z2,5,333_677_200302,"[14.46768, 49.99496]",U546Z2,Ke Březině,4.1,67.0,Wednesday,2020-05-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,8:15:00,8:15:00,16.22898,U749Z2,24,333_1012_200307,"[14.45197, 50.02019]",U749Z2,Tempo,16.3,23.0,Wednesday,2020-05-06
463,8:16:00,8:16:00,16.56781,U656Z2,25,333_1012_200307,"[14.45149, 50.02318]",U656Z2,Sídliště Krč,16.5,77.0,Wednesday,2020-05-06
464,8:17:00,8:17:00,17.03180,U893Z4,26,333_1012_200307,"[14.45143, 50.02735]",U893Z4,Zálesí,16.9,54.0,Wednesday,2020-05-06
465,8:19:00,8:19:00,17.58466,U488Z7,27,333_1012_200307,"[14.45515, 50.03038]",U488Z7,Nemocnice Krč,17.5,71.0,Wednesday,2020-05-06


In [77]:
# vybrani sloupce trip_id s ujetymi km
stop_dist = info_stops_clean.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED']]
stop_dist

,TRIP_ID,SHAPE_DIST_TRAVELED
0,333_677_200302,0.00000
1,333_677_200302,0.15836
2,333_677_200302,0.77992
3,333_677_200302,3.53721
4,333_677_200302,3.91690
...,...,...
462,333_1012_200307,16.22898
463,333_1012_200307,16.56781
464,333_1012_200307,17.03180
465,333_1012_200307,17.58466


In [73]:
# info_stops_clean['NEJBLIZSI_POZICE'] = 0.0
# info_stops_clean['DELAY_STOP_DEPARTURE'] = 0.0
# info_stops_clean['DAY_OF_WEEK'] = ''
# info_stops_clean['DATE'] = ''

In [67]:
# list unikatnich unicornu z dataframe data (allpositions)
unicorn_list = ranni_333['UNICORN'].unique().tolist()
len(unicorn_list)

#unicorn_try = data[data['UNICORN'] == '333_1052_200302_06_May'].sort_values(by='SHAPE_DIST_TRAVELED')

195

In [83]:
# prazdny novy DataFrame
final_tab = pd.DataFrame([], 
                         columns = ['UNICORN','DAY_OF_WEEK','TRIP_ID','STOP_ID','STOP_SEQUENCE','DELAY_STOP_DEPARTURE'])
final_tab

,UNICORN,DAY_OF_WEEK,TRIP_ID,STOP_ID,STOP_SEQUENCE,DELAY_STOP_DEPARTURE


In [82]:
# 195 unicornu -> pro kazdeho unicorna vybrat jen sloupecky co me zajimaji
# najit ten spravny jizdni rad trip_id == trip_id
# a na jeho zaklade priradit (zkratit) pocet pozic jen na pocet zastavek
for uni in unicorn_list:
    unicorn_find = ranni_333[ranni_333['UNICORN'] == uni]
    bus_dist = unicorn_find.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','DELAY_STOP_DEPARTURE']]
    
    for i, distance in enumerate(stop_dist['SHAPE_DIST_TRAVELED']):
        
        if stops_dist['TRIP_ID'] == bus_dist['TRIP_ID']:
        
            # vypocita absolutni hodnotu rozdilu vzdalenosti
            distances = abs(bus_dist['SHAPE_DIST_TRAVELED'] - distance)
            
            # najde nejmensi hodnotu v rozdilech
            row_min = distances.argmin()
            
            # lokace nejmensi hodnoty zpet v dataframu
            loc_row_min = bus_dist.iloc[row_min]
            
            #ukladat to do noveho dataframe
            info_stops_clean.loc[i,'NEJBLIZSI_POZICE'] = loc_row_min['SHAPE_DIST_TRAVELED']
            info_stops_clean.loc[i,'DELAY_STOP_DEPARTURE'] = loc_row_min['DELAY_STOP_DEPARTURE']
            info_stops_clean.loc[i,'DAY_OF_WEEK'] = loc_row_min['DAY_OF_WEEK']
            info_stops_clean.loc[i,'DATE'] = loc_row_min['DATE']
            # ulozit tento info_stops_clean do noveho dataframe a jet na dalsiho unicorna

In [ ]:
#     for position in bus_dist_333['SHAPE_DIST_TRAVELED']:

#         tatonejmensivzdalenost = abs(distance - position)
#         print(tatonejmensivzdalenost)
        
#         if tatonejmensivzdalenost < nejmensivzdalenost:
            
#             nejmensivzdalenost = tatonejmensivzdalenost
#             #print(nejmensivzdalenost)
            
#             info_stops_clean.loc[i,'NEJBLIZSI_POZICE'] = position